In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import geopandas as gp
import io
import base64
from IPython.display import HTML
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import subprocess
import warnings
import imageio
warnings.filterwarnings(action='ignore')
plt.style.use('fivethirtyeight')

Thank you for reading my Kernel.
Please give us a lot of feedback. I want to study through you.

# Introduction

In this analysis, I am going to use barplot, pie chart, and dynamic plot to visualize.

# Importance of Visualization

Today, visualization, many analysts think. This has led to the emergence of many visualization tools and platforms, which can be used to build dashboards or use analysis to find new insights. <br>
I think the importance of visualization as follows. <br>

**First, you can see a large amount of data at a glance.** <br>

**Second, more accurate data analysis results can be derived than summary statistics.** <br>

**Third, it is an effective way to inform people who do not know the data analysis.** <br>



Unlike other visualizations, dynamic visualizations have the effect of showing trends over time. <br>
I'm going to challenge dynamic visualization this time. We're not good enough and we've never done it, so please give us a lot of help and feedback.

# Data Upload

In [ ]:
data = pd.read_csv("../input/forest-fires-in-brazil/amazon.csv",encoding="ISO-8859-1")

In [ ]:
data.head()

# Data Check

Before dynamic visualization, let's simply look at the columns of the data.

In [ ]:
print("year : ", data['year'].unique())

You can see that data exists from 1998 to 2017.

In [ ]:
print("state : ", data['state'].unique())

It can be seen that a total of 23 states exist.

In [ ]:
print("month : ", data['month'].unique())

Since the moon is in Latin, we will translate it into English that we are familiar with.

In [ ]:
month_map={'Janeiro': 'January', 'Fevereiro': 'February', 'Março': 'March', 'Abril': 'April', 'Maio': 'May',
          'Junho': 'June', 'Julho': 'July', 'Agosto': 'August', 'Setembro': 'September', 'Outubro': 'October',
          'Novembro': 'November', 'Dezembro': 'December'}
#mapping our translated months
data['month']=data['month'].map(month_map)
#checking the month column for the second time after the changes were made
data.month.unique()

# Simple Visualization

In [ ]:
# data
plt.figure(figsize = (15,10))
data[['state','number']].groupby(['state']).number.sum().sort_values(ascending=True).plot.barh(width=0.9,color=sns.color_palette('inferno',15))
plt.axhspan(21.5,22.5 ,facecolor='Blue', alpha=0.4) # hilight space
plt.title('Number of Fire')
plt.show()

The area with the most fires appears to be Mato Cross. There are many fires compared to other areas.

In [ ]:
#plot
f, ax = plt.subplots(1,2, figsize = (25,15))
data[data['state'] == 'Mato Grosso'][['month','number']].groupby(['month']).number.sum().plot.pie(autopct='%1.1f%%',explode=[0,0,0,0,0.2,0,0,0,0.1,0,0,0], shadow=True,ax=ax[0])
ax[0].set_title('Mato Cross Month Fire', size = 25)
data[['month','number']].groupby(['month']).number.sum().plot.pie(autopct='%1.1f%%',explode=[0,0,0,0,0,0.2,0,0,0,0,0.2,0], shadow=True,ax=ax[1])
ax[1].set_title('ALL Month Fire', size = 25)
plt.show()

What's interesting is that in the case of Mato Gross, there are the most fires in January and May.
On the other hand, overall, there is a high rate of fire in Octorber and July.
It is likely that there will be regional climate differences as well.
If data is given, we can analyze it in depth.

Now let's try dynamic visualization.

# Animation Visualization

First of all, we will proceed with dynamic visualization through the matplotlib, which is familiar to us.

In [ ]:
ani_d1 = data[['year','number']].groupby(['year']).number.sum().reset_index()

In [ ]:
def animate(i):
    data = ani_d1.iloc[:int(i+1)]
    p = sns.barplot(x=data['year'], y=data['number'], data=data, palette="rocket")
    p.tick_params(labelsize=17)
    plt.setp(p.lines,linewidth=4)
    
fig = plt.figure(figsize=(10,6))
plt.xlim()
plt.ylim(np.min(ani_d1)[1], np.max(ani_d1)[1] + 10000)
plt.xlabel('Year',fontsize=20)
plt.title('Number of Fire Per Year',fontsize=20)

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=20, interval=500, repeat=True)
ani.save('plot.gif', writer='imagemagick')
plt.close()

video = io.open('plot.gif', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<img src="data:image/gif;base64,{0}" type="gif" />'''.format(encoded.decode('ascii')))

I completed dynamic visualization very simply. <br>
Then, let's use the characteristics of the data a little bit more to draw.

First, you need to get data on the latitude and longitude of Brazil's State. <br>
I hope the data since we've got in the https://geodata.lib.berkeley.edu/catalog/stanford-ys298mq8577 
The visualization method was referred to here. < https://towardsdatascience.com/how-to-make-a-gif-map-using-python-geopandas-and-matplotlib-cd8827cefbc8 >

In [ ]:
geo_data = gp.read_file('../input/geo-brazil-data/geo_brazil_data.shp')
geo_data = geo_data.rename(columns = {'nome': 'state'})

In [ ]:
ani_d2 = data[['year','state','number']].groupby(['year','state']).number.sum().reset_index()
ani_d2_pivot = ani_d2.pivot_table(values='number', index=['state'], columns='year').reset_index()
ani_d2_final = geo_data.set_index('state').join(ani_d2_pivot.set_index('state'))
# ani_d2_pivot = ani_d2.pivot_table(values='number', index=['state'], columns='year')
# ani_d2_final = pd.merge(ani_d2_pivot, geo_data, on='state', how='inner')
# ani_d2_final.head()

Let's first draw only the data that matches the state of Brazil's geographical data and the state of our data. That's because 10 of the 21 states don't have geographic information.

In [ ]:
filename = []
vmin, vmax = 150, 300

for year in ani_d2_final.columns[1:]:
    
    # create map
    fig = ani_d2_final.plot(column=year, cmap='Oranges', figsize=(8,5), linewidth=0.8, edgecolor='0.8', vmin=vmin, vmax=vmax, legend=True, norm=plt.Normalize(vmin=vmin, vmax=vmax))
    fig.axis('off')
    fig.set_title('Fire In Brazil', fontdict={'fontsize': '15', 'fontweight' : '3'})
        
    fig.annotate(year, xy=(0.1, .225), xycoords='figure fraction', horizontalalignment='left', verticalalignment='top',fontsize=20)
    chart = fig.get_figure()
    chart.savefig(str(year) +'_fire.png', dpi=300)
    filename.append(str(year) +'_fire.png')
    plt.close()

In [ ]:
images = []
for filename in filename:
    images.append(imageio.imread(filename))
imageio.mimsave('fire.gif', images)

In [ ]:
video = io.open('fire.gif', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<img src="data:image/gif;base64,{0}" type="gif" />'''.format(encoded.decode('ascii')))

# Conclusion

There are some findings from dynamic visualization. <br>

**First, dynamic visualization of time rather than simple visualization can quickly catch changes over time.** <br>
I think this is a very good thing. 
This is not just because you see a picture, but because you can see the changes over time through a single picture. <br>

**Second, the methods of visualization vary.**<br>
One way now is not the answer. <br>
I have found many Google blogs and applied them here.<br>
Many people have thought differently about the same topic and have seen the code simplify accordingly.<br>

Although it is still far from sufficient, the challenge will not stop because I want to visualize it with new perspectives and methods!